# Mtoto News Childline Prediction Challenge.

## Introduction
The objective of this competition is to create a forecast model to predict the number of incoming calls that Childline Kenya will receive per hour per day.

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload

## Loading the data.
The data available is in three files;
1. train.csv - contains train data.
2. KenyaPublicHolidays2016.csv	- list of public holidays in 2016
3. WeatherNairobi2016.xls - weather details.

In [12]:
df_train = pd.read_csv('train.csv')


/home/muoki/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12,13,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df_train.head(2)

,calldate,cc_status,maincat,subcat1,casepriority,referal,caller_gender,caller_age,caller_county,child_age,child_gender,child_county,parent_age,parent_gender,parent_county,Abuser_Relationship,Neglector_Relationship,Perpetrator_Relationship,Physical abuser_Relationship
0,2016-01-01 07:26:47,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 07:32:30,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_train.shape

(135988, 19)

The first column is a date time  column we can split it into date and time separate columns.


In [22]:
df_train['date'] = pd.to_datetime(df_train['calldate'])

In [27]:
df_train['cdate'] = df_train['date'].dt.date
df_train['ctime'] = df_train['date'].dt.time

In [33]:
df = df_train.drop(['calldate', 'date'], axis=1)

In [35]:
df = df[['cdate', 'ctime','cc_status', 'maincat', 'subcat1', 'casepriority', 'referal',
       'caller_gender', 'caller_age', 'caller_county', 'child_age',
       'child_gender', 'child_county', 'parent_age', 'parent_gender',
       'parent_county', 'Abuser_Relationship', 'Neglector_Relationship',
       'Perpetrator_Relationship', 'Physical abuser_Relationship']]

In [37]:
df.head()

,cdate,ctime,cc_status,maincat,subcat1,casepriority,referal,caller_gender,caller_age,caller_county,child_age,child_gender,child_county,parent_age,parent_gender,parent_county,Abuser_Relationship,Neglector_Relationship,Perpetrator_Relationship,Physical abuser_Relationship
0,2016-01-01,07:26:47,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01,07:32:30,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01,07:44:31,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01,07:46:13,Closed,others,Unrelated to remit of Childline 116,Non Critical,NaN,Male,Adult,Nyeri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01,07:47:22,Closed,non-interventional,Blank call,Non Critical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
!ls

KenyaPublicHolidays2016.csv			 Sample_Submission.csv
Mtoto_News_Childline_Prediction_Challenge.ipynb  train.csv
NairobiSchoolDates2016.csv			 WeatherNairobi2016.xls
readme.md


## Visualization

## Modeling

## Results

## Conclusions